In [4]:
using SCYFI
using JLD2
using Base.Threads
using Plots
using ProgressMeter
using LinearAlgebra
using LsqFit
using Statistics

"""
A,W,h PLRNN parameters
heuristic algorithm of finding FP (Durstewitz 2017) extended to find all k cycles
We need to solve: (A+WD_k)*...*(A+WD_1) *z + [(A+WD_k)*...*(A+WD_2)+...+(A+WD)**1+1]*h = z
"""
function scy_fi(
    A:: Array, W:: Array, h:: Array, order:: Integer, found_lower_orders:: Array;
    outer_loop_iterations:: Union{Integer,Nothing}= nothing,
    inner_loop_iterations:: Union{Integer,Nothing} = nothing,
    savepath::String="",
    name::Int=1
     )
    dim = size(A)[1]
    cycles_found = Array[]
    eigvals =  Array[]
    outer_loop_iterations, inner_loop_iterations = set_loop_iterations(order, outer_loop_iterations, inner_loop_iterations)
    i = -1
    num_init=0
    
    plot_regions=plot(legend=false)
    plot_regions_2=plot(size=(800,500),legend=false)
    iter = Iterators.product([0:(2^dim-1) for j=1:order]...)
    p = Progress(length(iter))
    unique_numbers_pool = Vector{Int}()
    #Exhaustively initialize the regions, then let the barebones of scyfi run s.t. we get the reproducable scyfi trajectory for every linear region 
    for (idx, item) in collect(enumerate(iter))
        unique_number_linear_regions = Array[]
        i += 1
        relu_matrix_list,unique_number= construct_relu_matrix_list_exhaustive(dim,order,item)
        unique_number_linear_regions=vcat(unique_number_linear_regions,unique_number)
        color= [cgrad(:rainbow, [-10, 10])[z] for z ∈ range(0.0, 1.0, length = 2^dim)] #RGB(rand(3)...)
        #scatter!(plot_regions,[0],[unique_number],color=color)
        difference_relu_matrices = 1
        c = 0
        while c < inner_loop_iterations
            c += 1
            z_candidate = get_cycle_point_candidate(A, W, relu_matrix_list, h, order)
            
            if z_candidate !== nothing
                trajectory = get_latent_time_series(order, A, W, h, dim, z_0=z_candidate)
                trajectory_relu_matrix_list = Array{Bool}(undef, dim, dim, order)
                for j = 1:order
                    trajectory_relu_matrix_list[:,:,j] = Diagonal(trajectory[j].>0)
                end
                # get unique number of the linear region
                unique_number= get_unique_number_linear_regions(dim,order,trajectory_relu_matrix_list)
                #scatter!(plot_regions,[c],[unique_number],color=color)
                unique_number_linear_regions=vcat(unique_number_linear_regions,unique_number)
                for j = 1:order
                    difference_relu_matrices = sum(abs.(trajectory_relu_matrix_list[:,:,j].-relu_matrix_list[:,:,j]))
                    if difference_relu_matrices != 0
                        break
                    end
                    if !isempty(found_lower_orders)
                        if map(temp -> round.(temp, digits=2), trajectory[1]) ∈ map(temp -> round.(temp,digits=2),collect(Iterators.flatten(Iterators.flatten(found_lower_orders))))
                            difference_relu_matrices = 1
                            break
                        end
                    end
                end
                if difference_relu_matrices == 0
                    if map(temp1 -> round.(temp1, digits=2), trajectory[1]) ∉ map(temp -> round.(temp, digits=2), collect(Iterators.flatten(cycles_found)))
                        e = get_eigvals(A,W,relu_matrix_list,order)
                        push!(cycles_found,trajectory)
                        push!(eigvals,e)
                        # Add the number of this quadrant to the list s.t. all inner iterations are plotted with this and not just end in the middle of the plot_regions
                        #if inner_loop_iterations-c >0
                        #    unique_number_linear_regions=vcat(unique_number_linear_regions,ones(inner_loop_iterations-c-1).*(unique_number))
                        #end

                        #i=0
                        #c=0
                        
                        num_init+=1
                    end
                end
                #if relu_matrix_list == trajectory_relu_matrix_list

                #    relu_matrix_list,unique_number = construct_relu_matrix_list(dim, order)
                #else
                relu_matrix_list = trajectory_relu_matrix_list
                #end
            #else
             #   relu_matrix_list,unique_number = construct_relu_matrix_list(dim, order)
            end 
        end
        plot!(plot_regions_2,0:inner_loop_iterations,unique_number_linear_regions,color=color[idx],alpha=0.1,ylabel="linear region")
        unique_numbers_pool =vcat(unique_numbers_pool, [unique_number_linear_regions])
        #save(savepath*"linear_regions_init_$idx"*"_$i.jld2","linear_regions",unique_number_linear_regions)
        
        next!(p)
    end
    #println(unique_numbers_pool)
    unique_numbers_arr = reduce(hcat, unique_numbers_pool)
    #Todo: how to do this without loop??
    num_unique = zeros(size(unique_numbers_arr)[1])
    for k = 1:size(unique_numbers_arr)[1]
        num_unique[k]=size(unique(transpose(unique_numbers_arr[k,:])))[1]
    end
    unique_last_5 = transpose(unique_numbers_arr[end-5,:])
    for k = 1:4
        println(size(unique_last_5))
        unique_last_5 = hcat(unique_last_5,transpose(unique_numbers_arr[end-k,:]))
    end
    unique_num_last_5 = size(unique(unique_last_5))[1]
    #println(num_unique)
    #add marker
    plot_regions=plot(collect(0:size(unique_numbers_arr)[1]-1),num_unique,label="data",linewidth=8,xguidefontsize=30,yguidefontsize=30,xtickfontsize=30,ytickfontsize=30,ylabel="#linear regions",xlabel="SCYFI iterations")
    # fit exponential decay curve with all parameters unknown
    m(t, p) = p[1] .* exp.(-p[2] * t).+p[3]
    p0 = [10000, 0.5,0.5]
    # fit 
    fit=curve_fit(m, collect(0:size(unique_numbers_arr)[1]-1), num_unique, p0)
    println(fit.param)
    #plot dashed
    #plot_regions=plot!(collect(0:size(unique_numbers_arr)[1]-1),m(collect(0:size(unique_numbers_arr)[1]-1),fit.param),line=(:dash, 8),label="fit with decay rate r=$(round(fit.param[2],digits=2))")
    # fit exponential decay curve with the assumption that the data already converged
    m2(t, p) = (2^dim - unique_num_last_5) .* exp.(-p[1] * t).+ unique_num_last_5
    p0 = [.5]
    # fit 
    fit2=curve_fit(m2, collect(0:size(unique_numbers_arr)[1]-1), num_unique, p0)
    println(fit2.param)
    #plot_regions=plot!(collect(0:size(unique_numbers_arr)[1]-1),m2(collect(0:size(unique_numbers_arr)[1]-1),fit2.param),linewidth=4,label="fit with decay rate (convergence assumed) r=$(round(fit2.param[1],digits=2))")
    #println(size(transpose(unique_numbers_arr)))
    #println(size(unique(unique_numbers_arr,dims=0)))
    plot_regions = plot(plot_regions_2,plot_regions,layout=(2,1),size=(1600,1000),xguidefontsize=30,yguidefontsize=30,xtickfontsize=30,ytickfontsize=30,margin=10Plots.mm,bottom_margin=20Plots.mm,legendfontsize=25)
    xlabel!("SCYFI iterations",fontsize=30)
    #ylabel!("linear region",fontsize=30)
    savefig(plot_regions,savepath*"linear regions_$name.png")
    #savefig(plot_regions_2,savepath*"linear regions2_$name.png")
    return cycles_found, eigvals,num_unique
end

a₁ = 0.7338
a₂ = 0.1774
w₁ = 1.8087
w₂ = -0.6787
h₁ = 0.1304
h₂ = 1.4858
A = [[a₁ 0]; [0 a₂]]
W = [[0 w₁]; [w₂ 0]]
h = [[h₁]; [h₂]]
order=1 

A= [0.8578 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.854 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0335 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.9136 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.9525 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.8768 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.1535 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.853 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.1409 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.1615]
W= [0.0 -1.4463 -0.337 1.7048 0.4761 1.7552 -1.9684 -1.3595 1.4374 -0.9695; -1.7675 0.0 1.927 1.5025 0.527 -0.3095 -0.1657 0.2703 0.8966 0.5809; -1.8445 1.4589 0.0 -1.0762 0.9967 -0.5734 0.3335 -0.2083 1.9636 1.9455; -1.4354 0.1999 0.1194 0.0 -1.9327 0.2794 -0.0638 -1.8584 1.2332 -0.9003; 1.3365 0.9002 -1.6608 -1.78 0.0 0.8564 0.0968 1.5093 -1.8805 -0.8452; 1.9984 -1.608 -0.003 -1.3368 1.6832 0.0 -0.6406 0.6705 1.3706 1.6651; -1.0215 1.5241 0.2583 -0.4604 -1.3028 0.4046 0.0 1.5681 1.9751 0.6568; 1.5711 -1.802 -1.9311 1.5438 0.2892 1.382 -0.7876 0.0 0.7347 1.042; -1.7086 0.1058 0.6493 -1.4929 -0.7276 -0.3506 0.9648 0.0739 0.0 1.4642; 1.7181 -1.3966 -0.4538 -0.5752 -1.3079 -1.7776 -0.0251 1.2183 -0.8343 0.0]
h=[0.1561, 0.8316, 0.6753, -0.2843, -0.3838, -1.7862, 1.8893, 0.1545, -1.9331, 1.8211]
#h=reshape(h)
cycles, eigenvals,num_unique0 = scy_fi(A, W, h, order,[],inner_loop_iterations=20,name=0)


# h_new=rand(-2.0:0.0001:2.0,10)

#Id=I
#W_new=rand(-2.0:0.0001:2.0,10,10)

#W_new=W_new-Diagonal(diag(W_new))

#A_new=Id+Diagonal(rand(-0.2:0.0001:0.2,10))
#dz=10
#A_new = Array(diagm(randn(dz)))
#W_ = randn(dz,dz)
#W_new =  Array(W_- Diagonal(W_))
#h_new = randn(dz)

A_new=[0.23595544021831238 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.22300233619099075 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.3794687147992607 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -2.2470245454322724 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.5568708132319689 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0153208005126833 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.8394793595454645 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.15032702970322 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0872214730746879 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.5737727051201574]
W_new= [0.0 0.030342735136971548 -1.960033105681724 -2.3565404993029833 -0.43839315565696835 0.4090887800238511 -1.0045349633281322 -0.776905744111141 0.5819505906561253 0.4843414298893679; 1.054489933821836 0.0 2.386353964799842 -0.25510922666279096 -0.8427136710445048 -1.56625562904541 -0.18489976345200224 0.17031972119274866 0.8205769423786091 -0.43091770589249906; -0.39928709086852476 0.9512413926914982 0.0 -0.2766029971214148 0.969847375151241 -0.07352383472184965 -1.9284511227211423 -1.613904964794951 -0.08101375513418892 -0.36071287778117556; -0.2953098840219087 0.5249697616003631 -0.2127473977023191 0.0 -0.35504043652464157 -0.6652450089694525 -0.4071649735237237 0.15425933427249994 1.0989325318367333 -0.2438454305169349; -0.056212412788616675 1.20193789957776 2.3000977641134654 -0.9474315088088849 0.0 0.6447962655207249 0.7638963983747441 0.2756717867609362 0.4173863120802429 -1.5949442934452602; -0.2629777418914434 -0.3131954603289703 -0.026795215666439484 0.06176417908073879 0.23895393054324907 0.0 -2.2764968514841097 -1.2443803547369645 0.6975492815416678 -0.7046160849831318; 0.5932154769863154 0.06428305730237636 2.1455784317418924 0.04970543264862137 1.4863259021970279 -0.507351878270895 0.0 1.6311399973052214 -0.810328731340883 1.8842958902488918; 0.2798141297226707 0.11890906163735358 1.1417739453756908 -1.3356626230616482 -0.5539590170054477 -0.8316179246944218 0.08279729662036586 0.0 -0.5983719155676963 -0.11928635578493116; -0.30523479635773204 -1.5460614793775194 -0.4064433701384464 -1.212900978748178 -0.3789190256286612 -0.5357933477717096 0.98899396375284 0.2615574492792097 0.0 -0.38747073813311556; -1.4807793618329783 1.3382129570871297 -0.49451044608346056 -0.34467175884175644 -0.9874885948772577 1.691170645586008 0.6710041398374672 0.17804892547338355 0.902489538961025 0.0]
h_new=[-1.5831339286065593, 0.29154013281062946, 1.3428354130733435, 0.8909185789954477, -0.8868327452818285, -0.38709276565240935, 0.3333205313848873, 1.2803042733162942, -0.1892995017656296, -1.760561230236739]
cycles, eigenvals,num_unique1 = scy_fi(A_new, W_new, h_new, order,[],inner_loop_iterations=20,savepath="",name=1)



A_new=[0.7199053843082656 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.5300618539068028 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.18248593331087207 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.12930055172453706 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.4241137242045743 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -1.2515868509758366 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.15187587379760065 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.2779057819431798 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.7290351664443059 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.438794897291892]
W_new= [0.0 -0.6779116531636149 1.0303407735094463 -0.3946985970565104 -1.3604188984718666 0.7517566313925257 0.5883186701929939 0.8562482714013175 -1.2508379692757319 0.9655642876293299; 0.4727298503015651 0.0 -0.10102067810735067 0.1277182442823283 1.3791610032449872 0.32065696099073043 0.3691239166788678 1.6024393423239638 -0.8473624334471769 -0.22685767247479555; -0.5161527256370043 0.4241028021234111 0.0 0.6261023090626524 0.9545279185989282 -0.5627038799490358 0.36417694091960756 -0.5631449236483892 -2.152252886529357 -0.10233809362198606; -0.014466004829569076 0.09831801677124424 -0.6872225046221723 0.0 0.8869296215421082 -0.33811386678590777 -0.06779592230261323 0.38718101255387205 -2.0622265227395675 0.5836207204468115; 1.1290492657093292 0.10321125812303812 -0.16525569000287713 -1.7399199715945706 0.0 0.7100992128271117 0.42374413716084286 -1.0553579563515607 -0.902240481317014 0.06209378319299482; -0.0613199115347654 1.441248140137618 -0.6115119970921327 -0.29356345057595934 1.478412947575989 0.0 1.5736855571003923 0.17419995102520253 -0.1619149940812472 -1.7261473284064108; 0.6480614103156072 0.8348292568529542 -0.9690555617197429 -0.9513341680421328 -1.1778986204222857 -1.2891458708462238 0.0 -1.250859096201562 -0.030385679763836642 -0.9246672524343529; 0.47964551359765517 0.7047646495287517 -0.4345007881873063 0.0354818819767976 -0.09402510020827845 -0.19046888939774942 -0.17660217935194328 0.0 0.3167082760001834 -0.1319081454675117; 0.8889862320671381 -0.1465979376850281 -0.8284497632094744 -1.4239234954828406 -0.46917961344262954 0.9373965895599665 -0.4723107183352553 0.5558605987249655 0.0 -0.6080338676347555; -0.46414754928587304 -0.43146427857292635 -1.500406582327295 -0.11572077169672068 -0.39706894431076645 -0.6153766141480758 1.3170278680875296 -1.2509109424522766 1.863792917410069 0.0]
h_new=[2.2062862756233006, 1.1123068019459486, -1.038926924105121, 1.1802451371008602, 0.9910935438008066, -0.13329064340666616, 0.38478632990993755, -0.7893463228094155, -1.786111860426036, 1.3596504945174464]
cycles, eigenvals,num_unique2 = scy_fi(A_new, W_new, h_new, order,[],inner_loop_iterations=20,savepath="",name=2)




A_new=[-0.17284576987994682 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.9284008411729292 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.1069676116986455 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.884582585953436 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -2.5191133120740252 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -2.0270562787951816 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -0.015207809316515357 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.30123599840857884 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.26868983577612054 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.27921573418081536]
W_new=[0.0 0.1840206840678437 0.9360948527819618 -1.5742988785654646 0.792203000274056 -0.3640810760046291 -1.5269065438534672 0.5544981241633811 0.3263526998939898 -0.9913458640131907; -0.36434551279060173 0.0 -0.16423583202703657 -1.5629234671198682 1.7304222346756317 -0.8707907746625635 0.8509718854839041 0.809291187812866 -0.07588190807339591 -0.8830634571999961; 3.0405650266748037 0.06951934382335381 0.0 0.4735383980439129 -0.57276120901401 -0.974840432290574 -0.865678027555077 0.4215600953898212 1.7243316303135787 1.5838946874645377; -1.7319591288259046 1.0699290195409914 -0.4811417062288991 0.0 -0.3024732407937885 0.763454869276352 0.28468446841076245 1.2550947305556057 0.32686363277291874 -1.465666891534617; -1.3683730240069671 -0.5750243044166377 0.6194294454148666 -1.699439351575357 0.0 -0.788165833256951 0.7450303005304022 -0.6491694036134421 0.3037017971304612 -0.1671201971193921; 0.42606023662931025 -2.307396347752808 -1.0966381171912807 0.8919991862563895 0.2371566810415793 0.0 -0.45288259625662564 0.19993094279464288 -1.6850839504246375 1.0243285544859702; -0.18949604346113463 -0.731491180558504 0.27420560223155005 2.0686583384854487 -0.7052118429486979 -0.9188715180187159 0.0 -1.2913729691091478 1.226971179292515 0.6980337352147014; -0.34756618342822565 -0.5541460939261365 -0.5234640390590872 0.35090677048234353 1.1399044739453645 0.36377934885769925 -0.6105382003121204 0.0 -0.23109929587106254 -0.9364743518262657; -0.7257158450997473 -1.236204653943488 0.1949593507745099 -0.872155065876195 0.9368412157265997 0.3864486293574672 1.274078975533499 -0.8308152620795684 0.0 1.05690406548524; -0.14942023282530809 -0.299026727888945 1.2627299468986917 0.8154847628861039 -1.0779699584167857 -1.1172780067566908 0.0721921829799658 -0.4621543005555036 0.7517551995108329 0.0]
h_new=[1.8500921880542003, 0.43026641860056053, 1.2891699980589628, 1.0387395231702352, 0.346535956959698, 0.8584385108604514, 0.8811816192985982, 0.1712890683773862, -1.3082119045579554, 1.661980528403806]
cycles, eigenvals,num_unique3 = scy_fi(A_new, W_new, h_new, order,[],inner_loop_iterations=20,savepath="",name=3)


A_new=[0.5839561044946332 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.21521264153690112 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.8816563299484811 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.09203403778377886 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -1.055339382156626 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.11765066356094471 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.5190882121337775 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.4715155752704758 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.15757850327717138 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.2463088998587849]
W_new=[0.0 -0.3457704414174833 -0.8664243541944705 -0.9924099044838655 -0.9664353433226844 0.6082462890114473 -0.3774127772818939 -1.0826421574386835 -1.4664417072133669 -0.005113928632663316; -2.031448540817121 0.0 -0.2976941543615429 -1.0545422924522858 1.6219275462786669 -0.43541239503730006 -1.7278160984250721 -0.02771376275732627 -0.281798488873464 1.7817579684935032; -0.7034445890207784 0.5263375677596415 0.0 0.6371717098795314 1.144788500336936 -0.3812005771049474 -1.9238455098958014 0.17458479996468454 1.0548457315618005 0.31670370344542054; -0.8863678234456819 1.8698187538039905 -0.3816598599780441 0.0 -0.6366062318251685 0.17739394518837154 0.8403543278640174 -1.0014852631376074 -0.7531218780111584 -0.05598983329582354; -1.0244819798955889 0.808514725907293 -1.3641979482444133 -0.2764179443236073 0.0 0.8850316068352887 -0.7968226407825296 0.2319594842677898 -0.2056712879006983 -0.7441489572655782; -0.006573470760159977 -1.2370952358906497 -1.4415292665317332 -0.9054874932661106 -0.46402636717176027 0.0 0.8085369047310926 0.3320537668021077 1.6244917711667297 0.9336114107649978; 0.2562642598451055 0.6188009833670736 0.4881271476817643 1.2954908497191224 -1.0576945092820875 1.0509509050465324 0.0 -0.1671125427401538 0.12553355617038056 -0.21702363390228185; 0.534238476542763 0.16495911891955295 -1.8590575213269684 -1.7228340225642709 0.4351242629034492 -1.2990969371483312 -0.9749125223685527 0.0 -0.08569036065720392 0.593882895623191; -0.21619190596903679 0.24165726702956603 0.9316434402231074 0.5802610361455469 1.2835487734017028 0.5146269372182436 1.3124968590581831 0.34077082125217534 0.0 0.48875511521129567; -1.9326498978499094 0.060650476039878956 -0.816978948281523 -0.8207299955204621 -0.08963355316905584 -0.02269133662915704 -1.6834534021848355 -1.7342518116424905 -0.2223359481409578 0.0]
h_new=[0.31869875873036496, -0.8771038554953012, 0.26560475747006074, 1.116745375947949, -0.47834437559454523, -0.060950467315760426, -1.044291340236365, 0.8188320211975282, 0.09514827654359592, -0.8004565796677684]
cycles, eigenvals,num_unique4 = scy_fi(A_new, W_new, h_new, order,[],inner_loop_iterations=20,savepath="",name=4)


#
A_new=[0.7770504278693248 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.1809428448527457 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.7047920456838375 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -1.327925566311366 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.6937518737773326 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.36805038872298595 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.007623176228811248 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.9569709351210207 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.674091728086139 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.089717593138288]
W_new=[0.0 0.08929264972404101 -0.8903723142929394 1.7876215822357977 1.3817261544104875 0.548589853072387 -0.7976723476885662 0.22604572532412895 -0.18632661105883253 0.04291064001409418; -0.2515917870727117 0.0 0.28174843647689696 -0.09104097092403657 0.07164010382108381 0.045724155620526105 0.4467795255271527 0.078583470632049 1.628988975991425 -0.8251379612729182; 0.47426376327282854 0.3326519570128778 0.0 0.435553370449495 -0.8193161685817367 3.074775399272723 0.13405080453514445 -0.12595581687400184 -0.5801654087143796 -0.12092802682666921; -0.9721382227732261 0.45451715101111384 0.9676419924663336 0.0 1.6079857537712379 0.06785823735540339 0.7980395616978954 0.7793052123819507 0.5651986241213678 0.4577901844893304; -0.3580904786963505 -0.06490858914346365 2.137747235451334 -0.9298985707280383 0.0 0.11254185664253376 -0.2971444692231096 -1.1384743569855251 -1.2515549125397352 -1.514872042490582; -0.1389434931308111 -1.3068319469335419 0.6918131881000187 1.4007748047809412 0.30580623745078445 0.0 0.6083108649718603 -0.33174151396579377 0.6431037974955125 -1.7497026246049578; -0.33263415941921237 0.042324072548759424 -1.102209472098761 -0.32527147549407703 1.8078409498803072 -0.7133007915516106 0.0 0.2574989398943863 0.365292485237477 0.5498945556572561; -0.724989667645116 -1.216613100150771 -1.351497049677286 1.7574120636099069 2.1132043859937193 -0.7218566959078427 -0.2804480364039024 0.0 2.236912389192855 0.987507334618776; -1.324593414785124 0.399040121385689 0.5286900634615166 -1.637170840457619 -1.0033241003446511 1.1026342168496477 0.6245010032340315 -0.43219913755822686 0.0 0.6997084781985804; 0.4033826925062086 0.5975814663000689 -1.0955998974921777 0.614899165008207 0.038739474729074734 -1.0959813187624066 -0.7602961013642373 -1.6876984141194715 -1.200149687204622 0.0]
h_new=[3.502113802782977, 1.5281674438551183, 1.3214665049397083, -0.644043695557827, 0.1745374913599884, 0.1750780236319685, 0.4281604336910176, -0.6770972934867514, -0.3249451206130187, 1.5543515457774488]
cycles, eigenvals,num_unique5 = scy_fi(A_new, W_new, h_new, order,[],inner_loop_iterations=20,savepath="",name=5)

A=  [0.13422427610206358 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.43158014018952445 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.7414748379000679 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.9003176498731927 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.4950478285859932 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.8905108837998785 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -1.1612408503371179 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.12578240436293867 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.5990512746189687 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.4443161909568998]
W=  [0.0 0.5377623697895649 0.010619718833321813 -0.0587474762253956 -0.3241574758710214 -0.012528026688212577 -0.7137123479795561 -0.4892514534778134 -0.798942366599385 -2.2232864199425597; -0.7851950869487452 0.0 0.6991306867655698 -1.8873508371298742 0.9820199097871601 -1.001893491554981 1.4789694277552032 0.7444198449692222 -1.3843948584589623 -0.6895250556834563; -0.5516361122102974 -1.4910367225758498 0.0 -0.4533948648481252 0.7840598463543382 -0.10607927669135976 -1.3501176476517167 0.5768709962721905 0.4930769614207714 -0.05198298878108091; -0.9988100235208338 1.310836033489006 1.3714682207501965 0.0 0.9033313751554088 -0.6583803423925499 -0.6215093491754184 0.9157799501025223 0.7371248445917806 -0.013628577528293735; -0.5168103477000577 0.025457880857131573 -0.6109280298108591 0.40699656690861435 0.0 1.2367005814757337 0.0935533260488156 1.3486670551981303 -1.3477334853207292 -1.3240265191949738; 2.2836971033256055 -1.2974594367081544 -0.90229279218769 0.05525269189680815 -0.04226436705227196 0.0 0.37760882013960845 -0.1899714228563672 0.08144730052101443 0.4863473983582797; -1.147438443831567 0.7905158812445437 -0.45044671483171533 -0.6026541873523937 0.04577161367332352 0.8609243023826254 0.0 1.9442986512497773 0.208217382793962 0.9560137567956818; -0.9939910592393684 -2.690954176155625 1.09862972524592 -1.2232151464261825 -0.5453098793248236 -0.8223249178181194 0.1265710116886458 0.0 0.13207971178640274 -0.9963794294819428; 0.26662672114814595 -0.08639596516064044 0.4606003583237791 -1.3536581275155866 -2.190369044174926 -2.0216688428520353 -0.2933924360994203 0.30611937521175475 0.0 -0.26948209542872603; 1.5215688972334436 -0.9187600449298773 -0.3122055258121508 -0.5786013920402098 -0.17684902495117352 -0.6994219193939029 1.0061645096809233 -0.3285856106221662 0.9141867509864171 0.0]
h=   [-0.2992305400979038, 1.237363215362512, -0.8895563127966278, 0.6987120544551962, -0.8215854907377209, 1.9977044064084153, 0.5090309193049556, -0.019298401795426363, 1.0723060729361826, 1.6758132986356278]
cycles, eigenvals,num_unique6 = scy_fi(A, W, h, order,[],inner_loop_iterations=20,savepath="",name=6)

A=  [0.8391341201502464 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -1.0811517152401386 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.8363170713531345 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.4974377492231528 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.14379809175786232 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6730767453693954 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.8446904566871831 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.20690519799098364 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.3094928629476899 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.3514969394130562]
W=  [0.0 -2.0868696031554195 0.4189363998478795 -0.9532899301998331 -0.6982464623032534 -0.18280355731865983 -0.5066170908242561 0.5172789359347645 0.6291659194298973 1.015255420879043; -0.6594321458418415 0.0 0.20441056525171122 -1.0426503171814165 -0.8065614182089564 -1.1293263385290349 0.41591669711865076 0.8498023823659223 0.11914707506345985 -1.568757774289476; -0.4802252368448371 0.4155086326795091 0.0 -0.9103224030487489 0.0470115941666211 1.0508999793552056 0.0783311822023618 -0.10968098709575916 -2.948500421842358 1.1008103659976534; -0.11223990880157919 1.1844898801304633 -0.19227589124824024 0.0 -0.9567438414173922 0.24000194775799236 0.2142858193541549 -0.5697846452593561 1.1446190950540869 -0.4234765579955127; 0.9621217962532872 0.1098148728033326 -1.0076714083212046 2.797286106175557 0.0 0.24978936438149346 0.1482519185038941 -0.7383287710855705 -1.8481212748255393 -0.8116470006275945; 0.3732923791384108 -1.2141331373909714 -0.8675676762877755 -0.596000423721703 0.47904459860555704 0.0 0.5740553964473688 0.38946158355390204 -0.5925410452756409 -0.694964999100404; 0.029020546294778137 -0.019891130904044436 -1.4579135655828537 -0.40659339084083473 -0.34143456170358727 0.8045191812742298 0.0 -0.5042472522988104 3.0756952080704596 -0.3145654680336291; -0.22846677269523297 -0.02620842855581849 -2.0303449680858994 1.1317298666386428 0.6625941644825086 0.1619972350770261 -0.32480380573225875 0.0 1.0812942231837386 -0.5014830218335378; -1.0077794185096254 2.0375143038450845 -0.3025894121478805 -0.42216799192441185 -1.0445550154041094 -0.6237715252702672 2.2417060107699585 -1.2862672018885295 0.0 -0.11345329708601162; -0.1414101527694233 0.5840014733217328 2.6267026633177344 0.5636450731434404 0.11532801784892809 0.41421716478232573 -0.039116052187156705 -0.5553075265469529 0.9214539143237405 0.0]
h=   [-0.6315200598166739, 1.6485770865630758, 0.08612985349143433, 1.090659890149538, -1.0363340595781092, -1.6454956836390227, -1.3514828630935045, 0.05181508179521258, -0.9586974663020497, 0.5419741045213575]
cycles, eigenvals,num_unique7 = scy_fi(A, W, h, order,[],inner_loop_iterations=20,savepath="",name=7)


A=  [-0.3714356290586279 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -1.225603045684025 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.5712741965141898 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.4218216838784457 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -2.8718386378022474 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.23272769565513407 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -1.499760614168128 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.3783420390176795 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.7551020898639108 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.7202646828969067]
W=  [0.0 -1.1135952552181587 1.6239166084497447 -0.04225448971183966 -2.3452765807080733 -0.8569205319639164 1.2270823429043394 -0.5614368426831728 2.0315105045202757 1.1924540675318003; 0.6156134626082599 0.0 2.331122256764964 -0.6226796921338232 0.9202069844625801 0.1677840526380118 1.6472148055008529 -1.0405022654811225 -3.0681841955145726 0.5932409541866397; 0.24422907149541054 -0.5178062131983797 0.0 0.5909095866405883 -0.004577296061738206 -1.6506443457474311 -0.38736453570073964 -1.6649217442341546 1.0768195566310061 0.4323671628753199; 0.07639826615525522 -1.0860365754849048 -2.4488557876211043 0.0 1.436659156731943 1.5236877301656344 1.0529893014887786 -1.6783029856536484 -0.5739760221399881 1.3512749130821105; 0.3402457200654621 0.4509340721606149 1.3678301235934307 -0.10295224967265186 0.0 0.5470193770051793 -0.2746189943210967 -1.0644416225755433 -0.07783275269763718 0.7846838911926598; 0.21390963184592277 0.1422309512473268 0.31961864396872514 0.23891785233168325 -0.621374235178144 0.0 -0.025091840038052378 1.2314718076677882 0.600854443517777 0.2582495222896048; 2.565932014584685 -0.331108200748869 0.3961980057120324 1.5684886032593337 0.52573387028556 1.535338372090091 0.0 0.7285153769899252 -1.0775169432446947 -0.5904855078576502; -0.575640316760484 0.6927130035384491 0.633086807244546 -0.5470974735955786 -0.7180889432448571 -0.5489164338822842 -1.375016304108414 0.0 -1.1487194319757303 -1.2558681584148295; 0.7217754227085899 -1.0100226241676238 1.0397171730522827 -1.1235841851060844 -0.6053380648344637 1.2262302917275905 -0.2590450878007165 -0.6093778981752188 0.0 0.5048457955104385; -0.802162951326631 -0.11653021771838767 -0.15000301106328506 0.5626260104479621 -0.15228578686621344 -0.46022578201038533 0.7865658950546657 -0.2251013470134919 1.1992629418440115 0.0]
h=   [-1.1418959539854359, 0.5462401328898079, 1.4890555382131065, -0.13102343696411656, 1.2017536876657193, 0.23116877593153184, 1.231624906339495, 0.21267184694026794, 0.025143067328729146, 0.3363740660862776]
cycles, eigenvals,num_unique8 = scy_fi(A, W, h, order,[],inner_loop_iterations=20,savepath="",name=8)


A=  [0.6927427549276983 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.8040841035142878 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.7827878506978986 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.5036253848857636 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.33010098070526706 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.3278259122079713 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -1.281705346987441 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.3443786529232924 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.14229132463954128 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.22045177163241636]
W=  [0.0 -0.08864247530863031 -1.7519426532676103 -1.6631675658508422 -1.975581115715138 -1.2940947677027792 -0.7077871314654253 0.16949613833006616 1.9335121753727071 0.8819405130753258; 0.23210663063051573 0.0 -1.0961676231911364 0.39793823422942387 1.1472729403622077 -0.19585981538526107 1.0021899558718759 -1.9863259918472072 0.870174113983614 0.9303462979855913; -0.8063685349841202 -0.15058445180348365 0.0 0.25095488539854244 -0.013795007582225747 -0.3728582480342313 1.055120588121979 0.33229952813552094 -0.8005115348114207 1.1868351542754298; -1.9961440674338407 1.7330570493732393 0.24872567796855813 0.0 0.6246765515025797 0.946006540495285 0.7525793708272149 0.4478290033174906 0.7418060650013795 0.8896920549813221; -0.5272809208832185 -1.0957582843414417 -0.523364434578797 -0.4676390110834428 0.0 0.7842422343764801 -0.2697190311336508 -1.3270352528057816 -0.6053283825014577 -0.37718181737156414; 0.47974585825968175 -0.9425626684602868 1.1788088611599405 -0.3953856265801734 0.014584065311158134 0.0 1.4664054234406623 -0.08903597352012182 0.42553725039266815 -0.2636413499747777; 0.509592901417285 -1.0265632799650402 -0.19100641439634045 0.1191435454672388 -0.89676796964786 0.13950299459012394 0.0 0.3133135637735069 1.9409432139494578 -2.0113682935181894; -0.5996206792891211 0.9818014441225593 -0.2057031342619579 0.5926879677111468 0.5802748216554412 0.5675775204472364 0.2660688868442082 0.0 -0.892976586472945 0.3600770335833809; -0.6337293256866156 0.5176697539658618 0.14215172578027183 -0.5634873747373561 -0.8012801830219756 0.49241018317483903 1.0020370134585541 -0.6489587691578198 0.0 -0.5468977869998611; 1.68082975728738 -0.18384515413112848 -0.7545412369772315 0.7734336156687788 1.5730212563041972 0.8981384050176521 -0.44357256148691804 1.4756794512227502 0.6300931252606276 0.0]
h=   [-0.37170941952894804, -1.2906582535465059, 0.5366038956411185, 0.24462811048702368, -0.46583871967454943, -0.7225630175310982, -0.4828063138104599, -0.15559809774398858, -0.6926416416837461, -1.170862483020094]
cycles, eigenvals,num_unique9 = scy_fi(A, W, h, order,[],inner_loop_iterations=20,savepath="",name=9)

println(mean(hcat(num_unique0,num_unique1,num_unique2,num_unique3,num_unique4,num_unique5,num_unique6,num_unique7,num_unique8,num_unique9),dims=2))
println(std(hcat(num_unique0,num_unique1,num_unique2,num_unique3,num_unique4,num_unique5,num_unique6,num_unique7,num_unique8,num_unique9),dims=2))


mean_=mean(hcat(num_unique0,num_unique1,num_unique2,num_unique3,num_unique4,num_unique5,num_unique6,num_unique7,num_unique8,num_unique9),dims=2)
std_=std(hcat(num_unique0,num_unique1,num_unique2,num_unique3,num_unique4,num_unique5,num_unique6,num_unique7,num_unique8,num_unique9),dims=2)
println(mean_-std_)
println(mean_+std_)
sum_plot=plot(collect(0:20),mean_, ribbon=(std_),legend=false,ylabel="mean #linear regions",xlabel="SCYFI iterations",linewidth=8,size=(1300,500),xguidefontsize=30,yguidefontsize=30,xtickfontsize=30,ytickfontsize=30,margin=10Plots.mm,bottom_margin=20Plots.mm,legendfontsize=25)
savefig(sum_plot,"summary_stat.png")
# A =[-0.7114138625078983 0.0; 0.0 -0.810071122299024]
# W = [0.0 -0.847123703434022; 1.0526999545024243 0.0]
# h = [1.143018234871081, -0.5677017359146972]
# cycles, eigenvals = scy_fi(A, W, h, 50,[],inner_loop_iterations=5,savepath="result/",name=50_1)


# A = [-1.1056077130580195 0.0; 0.0 -0.8087255327353905]
# W = [0.0 0.9370742103174913; -0.31485927342029074 0.0]
# h =  [-0.07371613901872398, 1.6300264304899281]
# cycles, eigenvals = scy_fi(A, W, h, 50,[],inner_loop_iterations=5,savepath="result/",name=50_2)


# A=  [-1.5951075737827023 0.0; 0.0 -1.3098557896538516]
# W=  [0.0 0.9091546789954821; 0.8477650700758326 0.0]
# h =  [1.1076806645871002, -0.43013320720566867]
# cycles, eigenvals = scy_fi(A, W, h, 50,[],inner_loop_iterations=5,savepath="result/",name=50_3)


# A  = [-0.7494469680118101 0.0; 0.0 -0.7557496558834026]
# W = [0.0 0.911308432653186; -1.3081937015754443 0.0]
# h =  [-0.015338274532312953, 1.3395405931240738]
# cycles, eigenvals = scy_fi(A, W, h, 50,[],inner_loop_iterations=5,savepath="result/",name=50_4)




Progress:   3%|██                                       |  ETA: 0:00:04

Progress:   8%|████                                     |  ETA: 0:00:03

Progress:  11%|█████                                    |  ETA: 0:00:03

Progress:  15%|███████                                  |  ETA: 0:00:03

Progress:  18%|████████                                 |  ETA: 0:00:03

...

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


(1, 1024)
(1, 2048)
(1, 3072)
(1, 4096)
[1008.9111605854389, 2.0372404628424836, 14.276883300368866]
[1.945376334174334

]
[1024.0; 163.5; 78.4; 51.2; 37.0; 28.7; 23.2; 19.5; 17.2; 15.4; 13.8; 12.8; 12.3; 11.9; 11.6; 11.3; 11.1; 10.9; 10.8; 10.7; 10.6;;]


[0.0; 55.355919085295454; 36.430146246694584; 26.98888660171071; 21.817424229271428; 17.52490291619963; 14.898172893195849; 12.642520845675254; 10.778579580713673; 9.252026804976303; 7.857056276579246; 7.450577904505872; 6.9290050592499295; 6.41958807955083; 6.186005711819758; 5.982381539896038; 6.045200116161949; 6.136412270663987; 6.088240030309799; 6.0562181085044955; 6.04060335470629;;]
[1024.0; 108.14408091470455; 41.96985375330542; 24.211113398289292; 15.182575770728572; 11.17509708380037; 8.30182710680415; 6.857479154324746; 6.421420419286326; 6.147973195023697; 5.942943723420755; 5.349422095494129; 5.370994940750071; 5.48041192044917; 5.413994288180242; 5.317618460103962; 5.054799883838051; 4.763587729336013; 4.711759969690202; 4.643781891495504; 4.55939664529371;;]
[1024.0; 218.85591908529545; 114.8301462466946; 78.18888660171072; 58.817424229271424; 46.224902916199625; 38.09817289319585; 32.14252084567525; 27.97857958071367; 24.652026804976302; 21.657056276579247; 20.2505779

"c:\\Users\\lukas.eisenmann\\Documents\\PhD\\Code\\SCYFI\\data\\FigureS9\\summary_stat.png"